In [1]:
# Importação das bibliotecas necessárias
import pandas as pd  
import win32com.client as win32
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager 

In [2]:
# Leitura da tabela de produtos a serem buscados no arquivo Excel
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)  # Exibe a tabela lida para conferência

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 16 128 gb,mini watch,5000,5500
1,playstation 5,slim,3000,4000
2,rtx 3060,zota galax,2500,3000


In [3]:
# Configura o serviço do ChromeDriver e inicializa o navegador Chrome
service = Service(ChromeDriverManager().install())
nav = webdriver.Chrome(service=service)

In [4]:
# Função para buscar ofertas no site Buscapé
def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    """
    Busca ofertas de um produto no site Buscapé, filtrando por termos banidos e faixa de preço.
    """
    # Acessa o site do Buscapé
    nav.get("https://www.buscape.com.br/")

    # Prepara os dados recebidos para facilitar a busca e comparação
    produto = produto.lower()  # Deixa o nome do produto em minúsculo
    termos_banidos = termos_banidos.lower()  # Deixa os termos banidos em minúsculo
    lista_termos_banidos = termos_banidos.split(" ")  # Separa os termos banidos em uma lista
    lista_termos_produto = produto.split(" ")  # Separa os termos do produto em uma lista
    preco_maximo = float(preco_maximo)  # Garante que o preço máximo é float
    preco_minimo = float(preco_minimo)  # Garante que o preço mínimo é float

    # Pesquisa o nome do produto no campo de busca do Buscapé
    nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)

    # Obtém a lista de resultados da busca
    lista_resultados = nav.find_elements(By.CLASS_NAME, "ProductCard_ProductCard_Inner__gapsh")

    # Lista para armazenar as ofertas encontradas
    lista_ofertas = []
    for resultado in lista_resultados:
        # Pega o nome do produto encontrado
        nome = resultado.find_element(By.CLASS_NAME, "ProductCard_ProductCard_Name__U_mUQ").text
        nome = nome.lower()

        # Verifica se o nome contém algum termo banido
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True

        # Verifica se o nome contém todos os termos do produto desejado
        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False

        # Se passou nas verificações, tenta extrair o preço e link
        if not tem_termos_banidos and tem_todos_termos_produto:
            try:
                # Extrai o preço do produto
                preco = resultado.find_element(By.CLASS_NAME, "Text_MobileHeadingS__HEz7L").text
                preco = (
                    preco.replace("R$", "")
                    .replace(" ", "")
                    .replace(".", "")
                    .replace(",", ".")
                )
                preco = float(preco)
                # Verifica se o preço está dentro do intervalo desejado
                if preco_minimo <= preco <= preco_maximo:
                    # Pega o link do produto
                    link = resultado.get_attribute("href")
                    lista_ofertas.append((nome, preco, link))
            except:
                # Se não conseguir extrair preço ou link, ignora o resultado
                continue
    return lista_ofertas  # Retorna a lista de ofertas encontradas

### Contrução da lista de ofertas encontradas

In [5]:
# Cria um DataFrame vazio para armazenar todas as ofertas encontradas
tabela_ofertas = pd.DataFrame()

# Percorre cada linha da tabela de produtos para buscar ofertas
for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]

    # Busca ofertas no Buscapé para o produto da linha atual
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        # Se encontrou ofertas, adiciona ao DataFrame principal
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape], ignore_index=True)
    else:
        tabela_buscape = None

display(tabela_ofertas)

,produto,preco,link
0,celular apple iphone 16 128gb,5248.99,https://www.buscape.com.br/celular/smartphone-...
1,playstation 5 edição digital 825gb,3850.00,https://www.buscape.com.br/console-de-video-ga...
2,placa de vídeo asus gaming - geforce dual rtx ...,2999.97,https://www.buscape.com.br/lead?oid=1255551101...
3,placa de video pcyes 12gb gpu nvidia gf rtx 30...,2699.00,https://www.buscape.com.br/lead?oid=1360057100...


### Exportar a base de ofertas para Excel

In [6]:
# Exporta a tabela de ofertas para um arquivo Excel
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

### Enviando o e-mail

In [7]:
# Se encontrou ofertas, envia um e-mail com a tabela no corpo do e-mail
if len(tabela_ofertas.index) > 0:
    # Cria um e-mail no Outlook
    outlook = win32.Dispatch("outlook.application")
    mail = outlook.CreateItem(0)
    mail.To = "joxawo8644@jeanssi.com"  # E-mail de destino
    mail.Subject = "Produto(s) Encontrado(s) na faixa de preço desejada"  # Assunto do e-mail
    # Corpo do e-mail em HTML com a tabela de ofertas
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att., Felipe Santos</p>
    """
    mail.Send()  # Envia o e-mailz

nav.quit()  # Encerra o navegador